# Forming features 

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import os
import sys
from geopy.geocoders import Nominatim
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.inspection import permutation_importance
from datetime import datetime, timedelta

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from datetime import timedelta

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.multioutput import MultiOutputRegressor

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
RAW_DATA_DIR = os.path.join(os.getcwd(), 'data', 'raw')
INTERIM_DATA_DIR = os.path.join(os.getcwd(), 'data', 'interim')

In [4]:
air_pollution_stations=['Helsinki Kallio 2',
 'Espoo Leppävaara Läkkisepänkuja',
 'Espoo Luukki',
 'Helsinki Mannerheimintie',
 'Vantaa Tikkurila Neilikkatie',
 'Helsinki Vartiokylä Huivipolku',
 'Vantaa Kehä III Viinikkala',
 'Helsinki Kustaa Vaasan tie']

weather_stations= [
 'Helsinki Vuosaari satama',
 'Helsinki Kaisaniemi',
 'Helsinki Kumpula',
 'Helsinki Malmi lentokenttä',
 'Sipoo Itätoukki']

forming database for air pollution

In [ ]:
# Loop through all air pollution stations
df_air_pollution_total = pd.DataFrame()

for station in air_pollution_stations:
    # Generate the filename for the station
    filename = f"{station.replace(' ', '_')}_air_pollution_data.parquet"
    full_path = os.path.join(RAW_DATA_DIR, filename)
    
    # Load the air pollution dataset for the station
    if os.path.exists(full_path):
        
        df_air_pollution = pd.read_csv(full_path)
        df_air_pollution['Timestamp'] = pd.to_datetime(df_air_pollution['Timestamp'])
        
        df_air_pollution.ffill(inplace=True)
        df_air_pollution.bfill(inplace=True)

        print(f"Loaded data for station: {station}")

        df_air_pollution = df_air_pollution[[ 'Timestamp', 'Particulate matter < 10 µm', 'Particulate matter < 2.5 µm', 'Nitrogen dioxide'  ]]       
                
        df_air_pollution = df_air_pollution.rename(columns= {'Particulate matter < 10 µm' : f'Particulate matter < 10 µm_{station}',
                                                             'Particulate matter < 2.5 µm' : f'Particulate matter < 2.5 µm_{station}', 
                                                             'Nitrogen dioxide' : f'Nitrogen dioxide_{station}'})

        if df_air_pollution_total.empty:
            df_air_pollution_total = df_air_pollution.copy()
        else:
            df_air_pollution_total = df_air_pollution_total.merge( df_air_pollution, how='outer', on='Timestamp' ) #, suffixes=('', f'_{station}'))        

    else:
        print(f"File not found for station: {station}")
        continue

df_air_pollution_total.ffill(inplace=True)

Loaded data for station: Helsinki Kallio 2
Loaded data for station: Espoo Leppävaara Läkkisepänkuja
Loaded data for station: Espoo Luukki
Loaded data for station: Helsinki Mannerheimintie
Loaded data for station: Vantaa Tikkurila Neilikkatie
Loaded data for station: Helsinki Vartiokylä Huivipolku
Loaded data for station: Vantaa Kehä III Viinikkala
Loaded data for station: Helsinki Kustaa Vaasan tie


In [7]:
df_weather_total = pd.DataFrame()

for station in weather_stations:
    # Generate the filename for the station
    filename = f"{station.replace(' ', '_')}_weather_data.csv"
    full_path = os.path.join(RAW_DATA_DIR, filename)

    # Load the weather dataset for the station
    if os.path.exists(full_path):
        df_weather = pd.read_csv(full_path)
        df_weather['Timestamp'] = pd.to_datetime(df_weather['Timestamp'])
        #print(f"Null value counts for station {station} before filling:")
        #print(df_weather.isnull().sum())
        df_weather.ffill( inplace=True)
        df_weather.bfill( inplace=True)
        #print(f"Loaded data for station: {station}")

        # Select relevant columns
        df_weather = df_weather[['Timestamp', "Air temperature", "Wind speed", "Gust speed"]]

        # Rename columns to include the station name (if you intend to track by station)
        df_weather = df_weather.rename(columns={
            "Air temperature": f"Air temperature_{station}",
            "Wind speed": f"Wind speed_{station}",
            "Gust speed": f"Gust speed_{station}"
        })

        if df_weather_total.empty:
            df_weather_total = df_weather.copy()
        else:
            df_weather_total = pd.merge(df_weather_total, df_weather, how='outer', on='Timestamp')

    else:
        print(f"File not found for station: {station}")
        continue

df_weather_total.ffill(inplace=True)
df_weather_total.bfill(inplace=True)

In [8]:
df_weather_total.to_csv(os.path.join(INTERIM_DATA_DIR, 'weather_data_total.csv'), index=False)
df_air_pollution_total.to_csv(os.path.join(INTERIM_DATA_DIR, 'air_pollution_data_total.csv'), index=False)